In [1]:
import pandas as pd

In [2]:
sales = pd.read_csv("./input_data/data/sales.csv")
sales.rename(columns={"store": "store_id", "product": "product_id", "quantity": "sales_product"}, inplace="True")

sales

,store_id,product_id,date,sales_product
0,0,0,2021-01-01,6
1,0,0,2021-01-02,19
2,0,0,2021-01-03,14
3,0,0,2021-01-04,10
4,0,0,2021-01-05,7
...,...,...,...,...
3745,4,4,2021-05-26,17
3746,4,4,2021-05-27,9
3747,4,4,2021-05-28,8
3748,4,4,2021-05-29,7


## Product Features

In [3]:
sales["MA7_P"] = sales.groupby(["store_id", "product_id"])["sales_product"].transform(lambda d: d.rolling(7, closed="left").mean())

sales

,store_id,product_id,date,sales_product,MA7_P
0,0,0,2021-01-01,6,NaN
1,0,0,2021-01-02,19,NaN
2,0,0,2021-01-03,14,NaN
3,0,0,2021-01-04,10,NaN
4,0,0,2021-01-05,7,NaN
...,...,...,...,...,...
3745,4,4,2021-05-26,17,14.428571
3746,4,4,2021-05-27,9,15.142857
3747,4,4,2021-05-28,8,13.714286
3748,4,4,2021-05-29,7,14.000000


In [4]:
sales["LAG7_P"] = sales.groupby(["store_id", "product_id"]).shift(periods=7)["sales_product"]

sales

,store_id,product_id,date,sales_product,MA7_P,LAG7_P
0,0,0,2021-01-01,6,NaN,NaN
1,0,0,2021-01-02,19,NaN,NaN
2,0,0,2021-01-03,14,NaN,NaN
3,0,0,2021-01-04,10,NaN,NaN
4,0,0,2021-01-05,7,NaN,NaN
...,...,...,...,...,...,...
3745,4,4,2021-05-26,17,14.428571,12.0
3746,4,4,2021-05-27,9,15.142857,19.0
3747,4,4,2021-05-28,8,13.714286,6.0
3748,4,4,2021-05-29,7,14.000000,14.0


In [5]:
# Checkpoint
f_1 = sales["product_id"] == 0
f_2 = sales["store_id"] == 0
sales.where(f_1 & f_2).head(10)

,store_id,product_id,date,sales_product,MA7_P,LAG7_P
0,0.0,0.0,2021-01-01,6.0,NaN,NaN
1,0.0,0.0,2021-01-02,19.0,NaN,NaN
2,0.0,0.0,2021-01-03,14.0,NaN,NaN
3,0.0,0.0,2021-01-04,10.0,NaN,NaN
4,0.0,0.0,2021-01-05,7.0,NaN,NaN
5,0.0,0.0,2021-01-06,6.0,NaN,NaN
6,0.0,0.0,2021-01-07,18.0,NaN,NaN
7,0.0,0.0,2021-01-08,10.0,11.428571,6.0
8,0.0,0.0,2021-01-09,10.0,12.000000,19.0
9,0.0,0.0,2021-01-10,3.0,10.714286,14.0


## Brand Aggregations

In [6]:
brand = pd.read_csv("./input_data/data/brand.csv")

brand.rename(columns={"name": "brand", "id": "brand_id"}, inplace=True)

brand

,brand,brand_id
0,Bartoletti Group,0
1,"Ryan, Pollich and Rolfson",1


In [7]:
product = pd.read_csv("./input_data/data/product.csv")

product.rename(columns={"id": "product_id"}, inplace=True)

product

,name,brand,product_id
0,Red Wine,Bartoletti Group,0
1,Water,Bartoletti Group,1
2,Smart Phone,"Ryan, Pollich and Rolfson",2
3,T-Shirt,Bartoletti Group,3
4,Laptop,"Ryan, Pollich and Rolfson",4


In [8]:
product = product.merge(brand, left_on="brand", right_on="brand")
product.drop("brand", inplace=True, axis=1)
product

,name,product_id,brand_id
0,Red Wine,0,0
1,Water,1,0
2,T-Shirt,3,0
3,Smart Phone,2,1
4,Laptop,4,1


In [9]:
sales_with_product = sales.join(product, on="product_id", rsuffix="_").drop(["product_id_", "name"], axis=1)

sales_with_product

,store_id,product_id,date,sales_product,MA7_P,LAG7_P,brand_id
0,0,0,2021-01-01,6,NaN,NaN,0
1,0,0,2021-01-02,19,NaN,NaN,0
2,0,0,2021-01-03,14,NaN,NaN,0
3,0,0,2021-01-04,10,NaN,NaN,0
4,0,0,2021-01-05,7,NaN,NaN,0
...,...,...,...,...,...,...,...
3745,4,4,2021-05-26,17,14.428571,12.0,1
3746,4,4,2021-05-27,9,15.142857,19.0,1
3747,4,4,2021-05-28,8,13.714286,6.0,1
3748,4,4,2021-05-29,7,14.000000,14.0,1


In [10]:
group_by = sales_with_product.groupby(["brand_id", "store_id", "date"])

group_by

In [11]:
ma7_b_sum = group_by["sales_product"].sum()

ma7_b = ma7_b_sum.transform(lambda d: d.rolling(7, closed="left").mean())

ma7_b

brand_id  store_id  date      
0         0         2021-01-01          NaN
                    2021-01-02          NaN
                    2021-01-03          NaN
                    2021-01-04          NaN
                    2021-01-05          NaN
                                    ...    
1         4         2021-05-26    24.000000
                    2021-05-27    25.428571
                    2021-05-28    24.000000
                    2021-05-29    24.142857
                    2021-05-30    23.285714
Name: sales_product, Length: 1500, dtype: float64

In [12]:
#Checkpoint
sales.where((sales["store_id"] == 0)&(sales["product_id"] == 3)&(sales["date"] == "2021-01-08")).dropna(how="all")

,store_id,product_id,date,sales_product,MA7_P,LAG7_P
457,0.0,3.0,2021-01-08,16.0,9.0,8.0


In [18]:
ma7_b_df = ma7_b.to_frame().rename(columns={"sales_product":"MA7_B"})
ma7_b_df["sales_brand"] = ma7_b_sum
ma7_b_df["LAG7_B"] = ma7_b_sum.shift(periods=7)
ma7_b_df

MA7_B  sales_brand  LAG7_B
brand_id store_id date                                      
0        0        2021-01-01        NaN           22     NaN
                  2021-01-02        NaN           29     NaN
                  2021-01-03        NaN           25     NaN
                  2021-01-04        NaN           23     NaN
                  2021-01-05        NaN           42     NaN
...                                 ...          ...     ...
1        4        2021-05-26  24.000000           28    18.0
                  2021-05-27  25.428571           14    24.0
                  2021-05-28  24.000000           26    25.0
                  2021-05-29  24.142857           10    16.0
                  2021-05-30  23.285714           23    31.0

[1500 rows x 3 columns]

In [15]:
#Checkpoint
f_1 = sales_with_product["store_id"] == 0
f_2 = sales_with_product["brand_id"] == 0
sales_with_product.where(f_1 & f_2).sort_values(["date", "store_id", "brand_id"]).head(5)

,store_id,product_id,date,sales_product,MA7_P,LAG7_P,brand_id
0,0.0,0.0,2021-01-01,6.0,NaN,NaN,0.0
150,0.0,1.0,2021-01-01,7.0,NaN,NaN,0.0
300,0.0,2.0,2021-01-01,9.0,NaN,NaN,0.0
1,0.0,0.0,2021-01-02,19.0,NaN,NaN,0.0
151,0.0,1.0,2021-01-02,8.0,NaN,NaN,0.0


In [16]:
b = sales_with_product.merge(ma7_b_df, on=["brand_id", "store_id", "date"])
b

,store_id,product_id,date,sales_product,MA7_P,LAG7_P,brand_id,MA7_B,sales_brand,LAG7_B
0,0,0,2021-01-01,6,NaN,NaN,0,NaN,22,NaN
1,0,1,2021-01-01,7,NaN,NaN,0,NaN,22,NaN
2,0,2,2021-01-01,9,NaN,NaN,0,NaN,22,NaN
3,0,0,2021-01-02,19,NaN,NaN,0,NaN,29,NaN
4,0,1,2021-01-02,8,NaN,NaN,0,NaN,29,NaN
...,...,...,...,...,...,...,...,...,...,...
3745,4,4,2021-05-28,8,13.714286,6.0,1,24.000000,26,25.0
3746,4,3,2021-05-29,3,10.142857,2.0,1,24.142857,10,16.0
3747,4,4,2021-05-29,7,14.000000,14.0,1,24.142857,10,16.0
3748,4,3,2021-05-30,14,10.285714,18.0,1,23.285714,23,31.0


In [17]:
b.where((b["store_id"] == 0)&(b["brand_id"] == 0)&(b["date"] == "2021-01-08")).dropna(how="all")

,store_id,product_id,date,sales_product,MA7_P,LAG7_P,brand_id,MA7_B,sales_brand,LAG7_B
21,0.0,0.0,2021-01-08,10.0,11.428571,6.0,0.0,28.571429,33.0,22.0
22,0.0,1.0,2021-01-08,14.0,8.142857,7.0,0.0,28.571429,33.0,22.0
23,0.0,2.0,2021-01-08,9.0,9.000000,9.0,0.0,28.571429,33.0,22.0


## Store Aggregations

In [42]:
date_store_sales = sales.groupby(["store_id", "date"])["sales_product"].sum()
ma7_s = date_store_sales.transform(lambda d: d.rolling(7, closed="left").mean())
ma7_s

store_id  date      
0         2021-01-01          NaN
          2021-01-02          NaN
          2021-01-03          NaN
          2021-01-04          NaN
          2021-01-05          NaN
                          ...    
4         2021-05-26    47.428571
          2021-05-27    48.000000
          2021-05-28    46.285714
          2021-05-29    46.142857
          2021-05-30    47.285714
Name: sales_product, Length: 750, dtype: float64

In [47]:
lag7_s = date_store_sales.shift(periods=7)
lag7_s

store_id  date      
0         2021-01-01     NaN
          2021-01-02     NaN
          2021-01-03     NaN
          2021-01-04     NaN
          2021-01-05     NaN
                        ... 
4         2021-05-26    48.0
          2021-05-27    53.0
          2021-05-28    45.0
          2021-05-29    30.0
          2021-05-30    59.0
Name: sales_product, Length: 750, dtype: float64

In [51]:
store_agg_df = ma7_s.to_frame().rename(columns={"sales_product": "MA7_S"})
store_agg_df["LAG7_S"] = lag7_s
store_agg_df["sales_store"] = date_store_sales
store_agg_df.head(10)

MA7_S  LAG7_S  sales_store
store_id date                                      
0        2021-01-01        NaN     NaN           39
         2021-01-02        NaN     NaN           48
         2021-01-03        NaN     NaN           53
         2021-01-04        NaN     NaN           29
         2021-01-05        NaN     NaN           57
         2021-01-06        NaN     NaN           32
         2021-01-07        NaN     NaN           64
         2021-01-08  46.000000    39.0           63
         2021-01-09  49.428571    48.0           41
         2021-01-10  48.428571    53.0           52

In [53]:
b

,store_id,product_id,date,sales_product,MA7_P,LAG7_P,brand_id,MA7_B,sales_brand,LAG7_B
0,0,0,2021-01-01,6,NaN,NaN,0,NaN,22,NaN
1,0,1,2021-01-01,7,NaN,NaN,0,NaN,22,NaN
2,0,2,2021-01-01,9,NaN,NaN,0,NaN,22,NaN
3,0,0,2021-01-02,19,NaN,NaN,0,NaN,29,NaN
4,0,1,2021-01-02,8,NaN,NaN,0,NaN,29,NaN
...,...,...,...,...,...,...,...,...,...,...
3745,4,4,2021-05-28,8,13.714286,6.0,1,24.000000,26,25.0
3746,4,3,2021-05-29,3,10.142857,2.0,1,24.142857,10,16.0
3747,4,4,2021-05-29,7,14.000000,14.0,1,24.142857,10,16.0
3748,4,3,2021-05-30,14,10.285714,18.0,1,23.285714,23,31.0


In [56]:
features_df = b.merge(store_agg_df, on=["store_id", "date"])
features_df

,store_id,product_id,date,sales_product,MA7_P,LAG7_P,brand_id,MA7_B,sales_brand,LAG7_B,MA7_S,LAG7_S,sales_store
0,0,0,2021-01-01,6,NaN,NaN,0,NaN,22,NaN,NaN,NaN,39
1,0,1,2021-01-01,7,NaN,NaN,0,NaN,22,NaN,NaN,NaN,39
2,0,2,2021-01-01,9,NaN,NaN,0,NaN,22,NaN,NaN,NaN,39
3,0,3,2021-01-01,8,NaN,NaN,1,22.428571,17,34.0,NaN,NaN,39
4,0,4,2021-01-01,9,NaN,NaN,1,22.428571,17,34.0,NaN,NaN,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,4,0,2021-05-30,2,7.571429,13.0,0,24.000000,17,28.0,47.285714,59.0,40
3746,4,1,2021-05-30,11,7.285714,7.0,0,24.000000,17,28.0,47.285714,59.0,40
3747,4,2,2021-05-30,4,9.142857,8.0,0,24.000000,17,28.0,47.285714,59.0,40
3748,4,3,2021-05-30,14,10.285714,18.0,1,23.285714,23,31.0,47.285714,59.0,40


## Features Dataframe Final Touchs

In [65]:
features_df = features_df[["product_id","store_id","brand_id","date","sales_product","MA7_P","LAG7_P","sales_brand","MA7_B","LAG7_B","sales_store","MA7_S","LAG7_S"]]
features_df.sort_values(by=["product_id","brand_id","store_id","date"], axis=0, inplace=True)
features_df.head(10)

,product_id,store_id,brand_id,date,sales_product,MA7_P,LAG7_P,sales_brand,MA7_B,LAG7_B,sales_store,MA7_S,LAG7_S
0,0,0,0,2021-01-01,6,NaN,NaN,22,NaN,NaN,39,NaN,NaN
5,0,0,0,2021-01-02,19,NaN,NaN,29,NaN,NaN,48,NaN,NaN
10,0,0,0,2021-01-03,14,NaN,NaN,25,NaN,NaN,53,NaN,NaN
15,0,0,0,2021-01-04,10,NaN,NaN,23,NaN,NaN,29,NaN,NaN
20,0,0,0,2021-01-05,7,NaN,NaN,42,NaN,NaN,57,NaN,NaN
25,0,0,0,2021-01-06,6,NaN,NaN,29,NaN,NaN,32,NaN,NaN
30,0,0,0,2021-01-07,18,NaN,NaN,30,NaN,NaN,64,NaN,NaN
35,0,0,0,2021-01-08,10,11.428571,6.0,33,28.571429,22.0,63,46.000000,39.0
40,0,0,0,2021-01-09,10,12.000000,19.0,19,30.142857,29.0,41,49.428571,48.0
45,0,0,0,2021-01-10,3,10.714286,14.0,38,28.714286,25.0,52,48.428571,53.0


In [66]:
features_df.to_csv("./output/features.csv")

In [163]:
wmape = features_df[["product_id", "store_id","brand_id","sales_product","MA7_P"]].copy()
wmape.dropna(inplace=True)
wmape["ape_dividing"] = abs(wmape["sales_product"]-wmape["MA7_P"])
wmape

,product_id,store_id,brand_id,sales_product,MA7_P,ape_dividing
35,0,0,0,10,11.428571,1.428571
40,0,0,0,10,12.000000,2.000000
45,0,0,0,3,10.714286,7.714286
50,0,0,0,7,9.142857,2.142857
55,0,0,0,2,8.714286,6.714286
...,...,...,...,...,...,...
3729,4,4,1,17,14.428571,2.571429
3734,4,4,1,9,15.142857,6.142857
3739,4,4,1,8,13.714286,5.714286
3744,4,4,1,7,14.000000,7.000000


In [171]:
ape_group = wmape.groupby(["product_id", "store_id", "brand_id"])[["ape_dividing", "sales_product"]]

In [174]:
df = ape_group.sum()
df

ape_dividing  sales_product
product_id store_id brand_id                             
0          0        0           692.857143           1241
           1        0           744.857143           1239
           2        0           727.285714           1409
           3        0           723.285714           1450
           4        0           754.142857           1336
1          0        0           922.000000           1301
           1        0           757.428571           1372
           2        0           746.857143           1337
           3        0           757.285714           1401
           4        0           714.142857           1400
2          0        0           793.714286           1309
           1        0           749.428571           1249
           2        0           714.428571           1343
           3        0           765.428571           1260
           4        0           684.285714           1363
3          0        1           716.571429           1433
           1        1           733.571429           1436
           2        1           775.857143           1230
           3        1           694.857143           1419
           4        1           765.000000           1323
4          0        1           809.142857           1346
           1        1           746.142857           1326
           2        1           811.285714           1382
           3        1           772.428571           1305
           4        1           819.142857           1420

In [190]:
wmape_df = df["ape_dividing"]/df["sales_product"]
wmape_df.to_frame("WMAPE").sort_values(["WMAPE"], ascending=False).to_csv("./output/mapes.csv")

Index(['WMAPE'], dtype='object')